<a href="https://colab.research.google.com/github/v-grand/Collector/blob/main/crypto_robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instal

In [1]:
!pip install ta
!pip install mplfinance

### Import

In [2]:

'''
%pip install prophet

'''
import requests
import urllib

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

import pandas as pd
from prophet import Prophet

from datetime import datetime, timedelta
import numpy as np
from pandas import DataFrame
import pandas as pd


from sklearn.preprocessing import MinMaxScaler
import ta
from typing import Tuple
import yfinance as yf

from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import mean_squared_error


from google.colab import drive
#from tensorflow.keras.models import load_model

from prophet import Prophet

# Подключение Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Function

In [3]:
def scale_data(data:DataFrame, columns_x, columns_y) -> Tuple[MinMaxScaler, np.ndarray, np.ndarray]:

    # Prepare target variable
    y = data[columns_y].values
    X = data [columns_x]

    # Scale features
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(y.reshape(-1, 1)).reshape(-1)

    return scaler, X_scaled, y_scaled

# EXTRACT

In [10]:
import yfinance as yf
import pandas as pd
import os

def extract_yf_data(ticker, periods, intervals):
    """
    Загружает данные для всех указанных интервалов и возвращает их в виде словаря.

    :param ticker: Символ тикера, например, 'BTC-USD'.
    :param periods: Период загрузки данных, например, '10y', 'max'.
    :param intervals: Список интервалов для загрузки данных, например, ['1m', '5m', '1d'].
    :return: Словарь с данными для каждого интервала.
    """
    datasets = {}

    for interval in intervals:
        try:
            print(f"Загрузка данных для интервала {interval}...")
            data = yf.download(tickers=ticker, period=periods, interval=interval)
            if data.empty:
                #print(f"Нет данных для интервала {interval}. Пропускаем.")
                continue
            else:
              # Добавляем столбец с интервалом для удобства
              #data['interval'] = interval

              # Сохраняем данные в словарь
              datasets[interval] = data
              print(f'{interval} ___ {datasets[interval].shape}')
              #print(datasets[interval].info())

        except Exception as e:
          pass
            #print(f"Ошибка при загрузке данных для интервала {interval}: {e}")

    print("Загрузка завершена.")
    return datasets




In [11]:
# Пример использования
ticker_ = 'BTC-USD'
period_ = 'max'
#intervals_ = ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']
intervals_ = ['1m','5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']



# Загрузка данных
datasets = extract_yf_data (ticker_, period_, intervals_)

Загрузка данных для интервала 1m...


[*********************100%***********************]  1 of 1 completed


1m ___ (7846, 5)
Загрузка данных для интервала 5m...


[*********************100%***********************]  1 of 1 completed


5m ___ (17256, 5)
Загрузка данных для интервала 15m...


...

1h ___ (17436, 5)
Загрузка данных для интервала 1d...
1d ___ (3783, 5)
Загрузка данных для интервала 5d...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


5d ___ (757, 5)
Загрузка данных для интервала 1wk...
1wk ___ (541, 5)
Загрузка данных для интервала 1mo...
1mo ___ (124, 5)
Загрузка данных для интервала 3mo...


[*********************100%***********************]  1 of 1 completed

3mo ___ (43, 5)
Загрузка завершена.


In [12]:
for dataset in datasets:
  print(dataset)
print(datasets[dataset].head())

1m
5m
15m
30m
60m
90m
1h
1d
5d
1wk
1mo
3mo
Price            Close        High         Low        Open      Volume
Ticker         BTC-USD     BTC-USD     BTC-USD     BTC-USD     BTC-USD
Date                                                                  
2014-09-01  378.046997  457.092987  289.295990  387.427002  1562727810
2014-12-01  254.263000  384.037994  171.509995  378.248993  2363432922
2015-03-01  230.190002  300.044006  214.873993  254.283005  2199559600
2015-06-01  230.056000  314.394012  199.567001  230.233002  2534864700
2015-09-01  377.321014  495.562012  225.117004  230.255997  3734526796


In [13]:
print(datasets[dataset].head())

Price            Close        High         Low        Open      Volume
Ticker         BTC-USD     BTC-USD     BTC-USD     BTC-USD     BTC-USD
Date                                                                  
2014-09-01  378.046997  457.092987  289.295990  387.427002  1562727810
2014-12-01  254.263000  384.037994  171.509995  378.248993  2363432922
2015-03-01  230.190002  300.044006  214.873993  254.283005  2199559600
2015-06-01  230.056000  314.394012  199.567001  230.233002  2534864700
2015-09-01  377.321014  495.562012  225.117004  230.255997  3734526796


In [14]:
print(datasets['1m'].info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7846 entries, 2025-01-17 14:59:00+00:00 to 2025-01-24 14:54:00+00:00
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (Close, BTC-USD)   7846 non-null   float64
 1   (High, BTC-USD)    7846 non-null   float64
 2   (Low, BTC-USD)     7846 non-null   float64
 3   (Open, BTC-USD)    7846 non-null   float64
 4   (Volume, BTC-USD)  7846 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 367.8 KB
None


# Robot

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class CryptoTradingBot:
    def __init__(self, initial_balance, dataset_path, ):
        self.balance = initial_balance
        self.positions = []
        self.dataset = pd.read_csv(dataset_path)
        self.dataset['signal'] = 0
        self.dataset['profit'] = 0

    def generate_signals(self):
        """Generate trading signals based on a moving average strategy."""
        short_window = 10
        long_window = 50

        self.dataset['short_ma'] = self.dataset['close'].rolling(window=short_window).mean()
        self.dataset['long_ma'] = self.dataset['close'].rolling(window=long_window).mean()

        self.dataset['signal'] = np.where(self.dataset['short_ma'] > self.dataset['long_ma'], 1, -1)

    def execute_trades(self):
        """Simulate trades based on generated signals."""
        for i in range(1, len(self.dataset)):
            signal = self.dataset['signal'].iloc[i]
            price = self.dataset['close'].iloc[i]

            if signal == 1:  # Buy signal
                self.positions.append(price)
            elif signal == -1 and self.positions:  # Sell signal
                entry_price = self.positions.pop(0)
                profit = price - entry_price
                self.balance += profit
                self.dataset.loc[i, 'profit'] = profit

    def backtest(self):
        """Run the backtest and calculate performance metrics."""
        self.generate_signals()
        self.execute_trades()

        total_profit = self.dataset['profit'].sum()
        win_rate = (self.dataset['profit'] > 0).mean()

        print(f"Initial Balance: {self.balance - total_profit:.2f}")
        print(f"Final Balance: {self.balance:.2f}")
        print(f"Total Profit: {total_profit:.2f}")
        print(f"Win Rate: {win_rate:.2%}")

    def plot_results(self):
        """Plot the results of the backtest."""
        plt.figure(figsize=(14, 7))
        plt.plot(self.dataset['close'], label='Close Price', alpha=0.7)
        plt.plot(self.dataset['short_ma'], label='Short Moving Average', alpha=0.7)
        plt.plot(self.dataset['long_ma'], label='Long Moving Average', alpha=0.7)

        plt.legend()
        plt.title("Trading Strategy Backtest")
        plt.show()

if __name__ == "__main__":
    dataset_path = "crypto_prices.csv"  # Replace with your dataset file
    initial_balance = 10000

    bot = CryptoTradingBot(initial_balance, dataset_path)
    bot.backtest()
    bot.plot_results()


FileNotFoundError: [Errno 2] No such file or directory: 'crypto_prices.csv'